In [1]:
from halotools.empirical_models import HodModelFactory
from halotools.empirical_models import TrivialPhaseSpace, ZuMandelbaum15Cens, ZuMandelbaum15Sats, \
                                        Leauthaud11Cens, Leauthaud11Sats, Zheng07Cens, Zheng07Sats, \
                                        NFWPhaseSpace, SubhaloPhaseSpace
from halotools.empirical_models import NFWPhaseSpace, SubhaloPhaseSpace, Tinker13Cens, Tinker13QuiescentSats, \
                                        TrivialProfile, Tinker13ActiveSats
from halotools_ia.ia_models.ia_model_components import CentralAlignment, RandomAlignment, RadialSatelliteAlignment, \
                                                        HybridSatelliteAlignment, MajorAxisSatelliteAlignment, SatelliteAlignment, \
                                                        SubhaloAlignment
from halotools_ia.ia_models.ia_strength_models import RadialSatelliteAlignmentStrengthAlternate

from halotools_ia.ia_models.nfw_phase_space import AnisotropicNFWPhaseSpace

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from halotools.mock_observables import tpcf

from halotools.sim_manager import HaloTableCache
from halotools.sim_manager import CachedHaloCatalog
from halotools_ia.correlation_functions import ed_3d, ee_3d

from halotools.utils import crossmatch

import time

import emcee
import corner
from IPython.display import display, Math

import warnings
warnings.filterwarnings("ignore")

In [2]:
##### FUNCTIONS

def get_specific_galaxy_coords(model, gal_type):
    x = model.mock.galaxy_table[model.mock.galaxy_table["gal_type"]==gal_type]["x"]
    y = model.mock.galaxy_table[model.mock.galaxy_table["gal_type"]==gal_type]["y"]
    z = model.mock.galaxy_table[model.mock.galaxy_table["gal_type"]==gal_type]["z"]
    return np.vstack( (x,y,z) ).T

def get_specific_galaxy_orientations(model, gal_type):
    axis_x = model.mock.galaxy_table[model.mock.galaxy_table["gal_type"]==gal_type]["galaxy_axisA_x"]
    axis_y = model.mock.galaxy_table[model.mock.galaxy_table["gal_type"]==gal_type]["galaxy_axisA_y"]
    axis_z = model.mock.galaxy_table[model.mock.galaxy_table["gal_type"]==gal_type]["galaxy_axisA_z"]
    return np.vstack( (axis_x,axis_y,axis_z) ).T

# Create mock catalogue instance
def create_mock(cens_occ_model, cens_prof_model, cens_orientation, sats_occ_model, 
                sats_prof_model, sats_prof_args, sats_orientation, central_alignment_strength, satellite_alignment_strength):
    # Set up the models
    mod = {}
    mod["cens_occ_model"] = cens_occ_model()
    mod["cens_prof_model"] = cens_prof_model()
    mod["cens_orientation"] = cens_orientation(central_alignment_strength=central_alignment_strength)
    mod["sats_occ_model"] = sats_occ_model()
    mod["sats_prof_model"] = sats_prof_model( *sats_prof_args )
    mod["sats_orientation"] = sats_orientation(satellite_alignment_strength=satellite_alignment_strength)
    
    model_instance = HodModelFactory(centrals_occupation = mod["cens_occ_model"],
                                     centrals_profile = mod["cens_prof_model"],
                                     satellites_occupation = mod["sats_occ_model"],
                                     satellites_profile = mod["sats_prof_model"],
                                     centrals_orientation = mod["cens_orientation"],
                                     satellites_orientation = mod["sats_orientation"],
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )
    
    # Populate the two mocks
    model_instance.populate_mock(halocat)
    print("number of galaxies: ", len(model_instance.mock.galaxy_table))
    
    return model_instance

def get_galaxy_coordinates_and_orientations(model_instance, halocat):
    galaxy_coords = np.vstack((model_instance.mock.galaxy_table['x'],
                                model_instance.mock.galaxy_table['y'],
                                model_instance.mock.galaxy_table['z'])).T
    galaxy_orientations = np.vstack((model_instance.mock.galaxy_table['galaxy_axisA_x'],
                                    model_instance.mock.galaxy_table['galaxy_axisA_y'],
                                    model_instance.mock.galaxy_table['galaxy_axisA_z'])).T
    
    # Account for nan values
    coord_nans = np.array( [ np.isnan( np.sum(val)) for val in galaxy_coords ] )
    orientation_nans = np.array( [ np.isnan( np.sum(val)) for val in galaxy_orientations ] )
    
    if np.sum(coord_nans) > 0:
        print("NaN rows in galaxy_coords: ", np.sum(coord_nans))
    if np.sum(orientation_nans) > 0:
        print("NaN rows in galaxy_orientations: ", np.sum(orientation_nans))
    
    all_nans = coord_nans | orientation_nans
    inds1, inds2 = crossmatch(all_nans, [True])
    mask = np.ones(len(all_nans), dtype=bool)
    mask[inds1] = False
    
    return galaxy_coords[mask], galaxy_orientations[mask]

def get_halo_coordinates_and_orientations(halocat, mass_thresh=10**11.9):
    mask = halocat.halo_table['halo_mpeak'] > mass_thresh
    halo_coords = np.vstack((halocat.halo_table['halo_x'],
                                halocat.halo_table['halo_y'],
                                halocat.halo_table['halo_z'])).T
    halo_orientations = np.vstack((halocat.halo_table['halo_axisA_x'],
                                    halocat.halo_table['halo_axisA_y'],
                                    halocat.halo_table['halo_axisA_z'])).T
    
    return halo_coords, halo_orientations, mask

def galaxy_alignment_correlations(galaxy_coords, galaxy_orientations, rbins):
    galaxy_omega = ed_3d( galaxy_coords, galaxy_orientations, galaxy_coords, rbins, period=halocat.Lbox )
    galaxy_eta = ee_3d(galaxy_coords, galaxy_orientations, galaxy_coords, galaxy_orientations, rbins, period=halocat.Lbox)
    galaxy_xi = tpcf(galaxy_coords, rbins, galaxy_coords, period=halocat.Lbox)
    
    return galaxy_omega, galaxy_eta, galaxy_xi

def halo_alignment_correlations(halo_coords, halo_orientations, mask, rbins):
    halo_omega = ed_3d(halo_coords[mask], halo_orientations[mask], halo_coords[mask],rbins, period=halocat.Lbox)
    halo_eta = ee_3d(halo_coords[mask], halo_orientations[mask], halo_coords[mask], halo_orientations[mask], rbins, period=halocat.Lbox)
    halo_xi = tpcf(halo_coords[mask], rbins, halo_coords[mask], period=halocat.Lbox)
    
    return halo_omega, halo_eta, halo_xi

def galaxy_cross_correlations(model_instance):
    central_coords = get_specific_galaxy_coords(model_instance, "centrals")
    central_orientations = get_specific_galaxy_orientations(model_instance, "centrals")
    satellite_coords = get_specific_galaxy_coords(model_instance, "satellites")
    satellite_orientations = get_specific_galaxy_orientations(model_instance, "satellites")
    
    # Account for nan values
    cen_coord_nans = np.array( [ np.isnan( np.sum(val)) for val in central_coords ] )
    cen_orientation_nans = np.array( [ np.isnan( np.sum(val)) for val in central_orientations ] )
    sat_coord_nans = np.array( [ np.isnan( np.sum(val)) for val in satellite_coords ] )
    sat_orientation_nans = np.array( [ np.isnan( np.sum(val)) for val in satellite_orientations ] )
    
    # Eliminate NaN values
    cen_nans = cen_coord_nans | cen_orientation_nans
    sat_nans = sat_coord_nans | sat_orientation_nans
    # For centrals
    inds1, inds2 = crossmatch(cen_nans, [True])
    mask = np.ones(len(cen_nans), dtype=bool)
    mask[inds1] = False
    central_coords = central_coords[mask]
    central_orientations = central_orientations[mask]
    # Now for satellites
    inds1, inds2 = crossmatch(sat_nans, [True])
    mask = np.ones(len(sat_nans), dtype=bool)
    mask[inds1] = False
    satellite_coords = satellite_coords[mask]
    satellite_orientations = satellite_orientations[mask]
    
    correlations = {}
    
    # Position-Shape
    correlations["cencen_omega"] = ed_3d(central_coords, central_orientations, central_coords, rbins, period=halocat.Lbox)
    correlations["censat_omega"] = ed_3d(central_coords, central_orientations, satellite_coords, rbins, period=halocat.Lbox)
    correlations["satsat_omega"] = ed_3d(satellite_coords, satellite_orientations, satellite_coords, rbins, period=halocat.Lbox)
    
    # Shape-Shape
    correlations["cencen_eta"] = ee_3d(central_coords, central_orientations, central_coords, central_orientations, rbins, period=halocat.Lbox)
    correlations["censat_eta"] = ee_3d(central_coords, central_orientations, satellite_coords, satellite_orientations, rbins, period=halocat.Lbox)
    correlations["satsat_eta"] = ee_3d(satellite_coords, satellite_orientations, satellite_coords, satellite_orientations, rbins, period=halocat.Lbox)
    
    # Position-Position
    cencen_xi, censat_xi, satsat_xi = tpcf(central_coords, rbins, satellite_coords, period=halocat.Lbox)
    correlations["cencen_xi"] = cencen_xi
    correlations["censat_xi"] = censat_xi
    correlations["satsat_xi"] = satsat_xi
    
    return correlations

# Eliminate halos with 0 for halo_axisA_x(,y,z)
def mask_bad_halocat(halocat):
    bad_mask = (halocat.halo_table["halo_axisA_x"] == 0) & (halocat.halo_table["halo_axisA_y"] == 0) & (halocat.halo_table["halo_axisA_z"] == 0)
    halocat._halo_table = halocat.halo_table[ ~bad_mask ]

In [3]:
cache = HaloTableCache()
for entry in cache.log: print(entry)

#halocat = CachedHaloCatalog(simname='multidark', redshift=0)
halocat = CachedHaloCatalog(simname='bolplanck', halo_finder='rockstar', redshift=0, version_name='halotools_v0p4')
mask_bad_halocat(halocat)

('bolplanck', 'rockstar', 'halotools_v0p4', '-0.0023', 'C:\Users\nvana\.astropy\cache\halotools\halo_catalogs\bolplanck\rockstar\hlist_1.00231.list.halotools_v0p4.hdf5')
('bolshoi', 'rockstar', 'halotools_v0p4', '-0.0003', 'C:\Users\nvana\.astropy\cache\halotools\halo_catalogs\bolshoi\rockstar\hlist_1.00035.list.halotools_v0p4.hdf5')


In [4]:
rbins = np.logspace(-1,1.4,20)
rbin_centers = (rbins[:-1]+rbins[1:])/2.0

a = 0.826
gamma = 0

# MODELS
cens_occ_model = Leauthaud11Cens
cens_prof_model = TrivialPhaseSpace
cens_orientation = CentralAlignment
sats_occ_model = Leauthaud11Sats
sats_prof_model1 = SubhaloPhaseSpace
prof_args1 = ("satellites", np.logspace(10.5, 15.2, 15))
#sats_orientation1 = SubhaloAlignment(satellite_alignment_strength=0.5, halocat=halocat)
sats_orientation1 = RadialSatelliteAlignment
#sats_orientation1 = SubhaloAlignment
if a is None or gamma is None:
    sats_strength = RadialSatelliteAlignmentStrengthAlternate()
else:
    sats_strength = RadialSatelliteAlignmentStrengthAlternate(satellite_alignment_a=a, satellite_alignment_gamma=gamma)
Lbox = halocat.Lbox
sats_strength.inherit_halocat_properties(Lbox=Lbox)

title1 = "Radially Dependent Radial Alignment Strength"
title2 = "Constant Radial Alignment Strength"

central_alignment = 1

In [5]:
# Initially create the mock with SubhaloAlignment to overwrite the host information with subhalo
model_instance = HodModelFactory(centrals_occupation = cens_occ_model(),
                                     centrals_profile = cens_prof_model(),
                                     satellites_occupation = sats_occ_model(),
                                     satellites_profile = sats_prof_model1(*prof_args1),
                                     satellites_radial_alignment_strength = sats_strength,
                                     centrals_orientation = cens_orientation(central_alignment_strength=central_alignment),
                                     satellites_orientation = sats_orientation1(satellite_alignment_strength=0.8, halocat=halocat),
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'satellites_radial_alignment_strength',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )

#model_instance.populate_mock(halocat)
#seed=132358712
seed=None
model_instance.populate_mock(halocat, seed=seed)
# Then replace the SubhaloAlignment with whatever we actually want to use
#model_instance._input_model_dictionary["satellites_orientation"] = sats_orientation1(satellite_alignment_strength=0.8, halocat=halocat)
model_instance._input_model_dictionary["satellites_orientation"].inherit_halocat_properties( Lbox=Lbox )


In [6]:
halo_etas = []
halo_omegas = []
halo_cen_omegas = []
halo_xis = []

galaxy_etas = []
galaxy_omegas = []
galaxy_xis = []

all_halo_etas = []
all_halo_omegas = []
all_halo_xis = []

all_galaxy_etas = []
all_galaxy_omegas = []
all_galaxy_xis = []

halo_diff_etas = []
halo_diff_omegas = []
halo_diff_xis = []

halo_sat_shape_cen_pos_omegas = []
halo_cen_shape_sat_pos_omegas = []
halo_satsat_omegas = []
halo_cencen_omegas = []
halo_cen_sat_etas = []
halo_cen_sat_xis = []

galaxy_sat_shape_cen_pos_omegas = []
galaxy_cen_shape_sat_pos_omegas = []
galaxy_satsat_omegas = []
galaxy_cencen_omegas = []
galaxy_cen_sat_etas = []
galaxy_cen_sat_xis = []

halo_coords, halo_orientations, mask = get_halo_coordinates_and_orientations(halocat)
runs = 100

#model_instance.populate_mock(halocat)

begin = time.time()

loop = 0

for i in range(runs):
    loop += 1
    
    #model_instance.populate_mock(halocat)
    model_instance.mock.populate()
    #model_instance._input_model_dictionary["satellites_orientation"].assign_satellite_orientation( table=model_instance.mock.galaxy_table )
    #model_instance._input_model_dictionary["centrals_orientation"].assign_central_orientation( table=model_instance.mock.galaxy_table )
    print("number of galaxies: ", len(model_instance.mock.galaxy_table))
    
    sat_cut = model_instance.mock.galaxy_table[ model_instance.mock.galaxy_table["gal_type"] == "satellites" ]
    cen_cut = model_instance.mock.galaxy_table[ model_instance.mock.galaxy_table["gal_type"] == "centrals" ]
    
    # Get all halo coords and then get halo coords and orientations for just subhalos
    # First, all halo coords
    inds1, inds2 = crossmatch(model_instance.mock.galaxy_table['halo_id'], halocat.halo_table['halo_id'])
    halo_coords_all = np.array( [ halocat.halo_table[inds2]["halo_x"], 
                                halocat.halo_table[inds2]["halo_y"],
                                halocat.halo_table[inds2]["halo_z"] ] ).T
    halo_orientations_all = np.array( [ halocat.halo_table[inds2]["halo_axisA_x"], 
                                halocat.halo_table[inds2]["halo_axisA_y"],
                                halocat.halo_table[inds2]["halo_axisA_z"] ] ).T
    # Now get just the subhalo information
    inds1, inds2 = crossmatch(sat_cut['halo_id'], halocat.halo_table['halo_id'])
    halo_coords_sat = np.array( [ halocat.halo_table[inds2]['halo_x'],
                                halocat.halo_table[inds2]['halo_y'],
                                halocat.halo_table[inds2]['halo_z'] ] ).T
    halo_orientations_sat = np.array( [ halocat.halo_table[inds2]['halo_axisA_x'],
                                    halocat.halo_table[inds2]['halo_axisA_y'],
                                    halocat.halo_table[inds2]['halo_axisA_z'] ] ).T
    
    # Get central halo info
    inds1, inds2 = crossmatch(cen_cut['halo_id'], halocat.halo_table['halo_id'])
    halo_coords_cen = np.array( [ halocat.halo_table[inds2]['halo_x'],
                                halocat.halo_table[inds2]['halo_y'],
                                halocat.halo_table[inds2]['halo_z'] ] ).T
    halo_orientations_cen = np.array( [ halocat.halo_table[inds2]['halo_axisA_x'],
                                    halocat.halo_table[inds2]['halo_axisA_y'],
                                    halocat.halo_table[inds2]['halo_axisA_z'] ] ).T
    
    # Now do the same for the galaxy information
    galaxy_coords_all = np.array( [ model_instance.mock.galaxy_table["x"], 
                                    model_instance.mock.galaxy_table["y"],
                                    model_instance.mock.galaxy_table["z"] ] ).T
    galaxy_orientations_all = np.array( [ model_instance.mock.galaxy_table["galaxy_axisA_x"], 
                                    model_instance.mock.galaxy_table["galaxy_axisA_y"],
                                    model_instance.mock.galaxy_table["galaxy_axisA_z"] ] ).T
    galaxy_coords_sat = np.array( [ sat_cut['x'],
                                    sat_cut['y'],
                                    sat_cut['z'] ] ).T
    galaxy_orientations_sat = np.array( [ sat_cut['galaxy_axisA_x'],
                                        sat_cut['galaxy_axisA_y'],
                                        sat_cut['galaxy_axisA_z'] ] ).T
    
    galaxy_coords_cen = np.array( [ cen_cut['x'],
                                    cen_cut['y'],
                                    cen_cut['z'] ] ).T
    galaxy_orientations_cen = np.array( [ cen_cut['galaxy_axisA_x'],
                                        cen_cut['galaxy_axisA_y'],
                                        cen_cut['galaxy_axisA_z'] ] ).T
    
    # Get the halo information for just the satellite halos in model_instance1
    #cut = model_instance.mock.galaxy_table[ model_instance.mock.galaxy_table["gal_type"] == "satellites" ]
    #inds1, inds2 = crossmatch(cut['halo_id'], halocat.halo_table['halo_id'])
    #used_mask1 = np.zeros(len(halocat.halo_table), dtype=bool)
    #used_mask1[inds2] = True
    #halo_omega, halo_eta, halo_xi = halo_alignment_correlations(halo_coords, halo_orientations, used_mask1, rbins)
    #galaxy_coords = get_specific_galaxy_coords(model_instance1, "satellites")
    #galaxy_orientations = get_specific_galaxy_orientations(model_instance1, "satellites")
    #galaxy_omega, galaxy_eta, galaxy_xi = galaxy_alignment_correlations(galaxy_coords, galaxy_orientations, rbins)
    
    # Calculate the correlation functions
    #halo_omega = ed_3d( halo_coords_sat, halo_orientations_sat, halo_coords_all, rbins, period=halocat.Lbox )
    #halo_cen_omega = ed_3d( halo_coords_cen, halo_orientations_cen, halo_coords_all, rbins, period=halocat.Lbox )
    #halo_eta = ee_3d(halo_coords_sat, halo_orientations_sat, halo_coords_sat, halo_orientations_sat, rbins, period=halocat.Lbox)
    #halo_xi = tpcf(halo_coords_all, rbins, halo_coords_all, period=halocat.Lbox)
    
    #galaxy_omega = ed_3d( galaxy_coords_sat, galaxy_orientations_sat, galaxy_coords_all, rbins, period=halocat.Lbox )
    #galaxy_eta = ee_3d(galaxy_coords_sat, galaxy_orientations_sat, galaxy_coords_sat, galaxy_orientations_sat, rbins, period=halocat.Lbox)
    #galaxy_xi = tpcf(galaxy_coords_all, rbins, galaxy_coords_all, period=halocat.Lbox)
    
    # All
    #all_halo_omega = ed_3d( halo_coords_all, halo_orientations_all, halo_coords_all, rbins, period=halocat.Lbox )
    #all_halo_eta = ee_3d(halo_coords_all, halo_orientations_all, halo_coords_all, halo_orientations_all, rbins, period=halocat.Lbox)
    #all_halo_xi = tpcf(halo_coords_all, rbins, halo_coords_all, period=halocat.Lbox)
    
    all_galaxy_omega = ed_3d( galaxy_coords_all, galaxy_orientations_all, galaxy_coords_all, rbins, period=halocat.Lbox )
    all_galaxy_eta = ee_3d(galaxy_coords_all, galaxy_orientations_all, galaxy_coords_all, galaxy_orientations_all, rbins, period=halocat.Lbox)
    all_galaxy_xi = tpcf(galaxy_coords_all, rbins, galaxy_coords_all, period=halocat.Lbox)
    
    # Cross information
    ##halo_sat_shape_cen_pos_omega = ed_3d( halo_coords_sat, halo_orientations_sat, halo_coords_cen, rbins, period=halocat.Lbox )
    #halo_cen_shape_sat_pos_omega = ed_3d( halo_coords_cen, halo_orientations_cen, halo_coords_sat, rbins, period=halocat.Lbox )
    #halo_cen_sat_eta = ee_3d(halo_coords_sat, halo_orientations_sat, halo_coords_cen, halo_orientations_cen, rbins, period=halocat.Lbox)
    #halo_satsat_omega = ed_3d( halo_coords_sat, halo_orientations_sat, halo_coords_sat, rbins, period=halocat.Lbox )
    #halo_cencen_omega = ed_3d( halo_coords_cen, halo_orientations_cen, halo_coords_cen, rbins, period=halocat.Lbox )
    #halo_cen_sat_xi = tpcf(halo_coords_B, rbins, halo_coords_cen, period=halocat.Lbox)
    
    ##galaxy_sat_shape_cen_pos_omega = ed_3d( galaxy_coords_sat, galaxy_orientations_sat, galaxy_coords_cen, rbins, period=halocat.Lbox )
    #galaxy_cen_shape_sat_pos_omega = ed_3d( galaxy_coords_cen, galaxy_orientations_cen, galaxy_coords_sat, rbins, period=halocat.Lbox )
    #galaxy_cen_sat_eta = ee_3d(galaxy_coords_sat, galaxy_orientations_sat, galaxy_coords_cen, galaxy_orientations_cen, rbins, period=halocat.Lbox)
    #galaxy_satsat_omega = ed_3d( galaxy_coords_sat, galaxy_orientations_sat, galaxy_coords_sat, rbins, period=halocat.Lbox )
    #galaxy_cencen_omega = ed_3d( galaxy_coords_cen, galaxy_orientations_cen, galaxy_coords_cen, rbins, period=halocat.Lbox )
    #galaxy_cen_sat_xi = tpcf(galaxy_coords_B, rbins, galaxy_coords_cen, period=halocat.Lbox)
    
    # Store the results
    #halo_omegas.append( halo_omega )
    #halo_cen_omegas.append( halo_cen_omega )
    #halo_etas.append( halo_eta )
    #halo_xis.append( halo_xi )
    
    #galaxy_omegas.append( galaxy_omega )
    #galaxy_etas.append( galaxy_eta )
    #galaxy_xis.append( galaxy_xi )
    
    #all_halo_omegas.append( all_halo_omega )
    #all_halo_etas.append( all_halo_eta )
    #all_halo_xis.append( all_halo_xi )
    
    all_galaxy_omegas.append( all_galaxy_omega )
    all_galaxy_etas.append( all_galaxy_eta )
    all_galaxy_xis.append( all_galaxy_xi )
    
    #halo_sat_shape_cen_pos_omegas.append( halo_sat_shape_cen_pos_omega )
    #halo_cen_shape_sat_pos_omegas.append( halo_cen_shape_sat_pos_omega )
    #halo_satsat_omegas.append( halo_satsat_omega )
    #halo_cencen_omegas.append( halo_cencen_omega )
    #halo_cen_sat_etas.append( halo_cen_sat_eta )
    #halo_cen_sat_xis.append( halo_cen_sat_xi )
    
    ##galaxy_sat_shape_cen_pos_omegas.append( galaxy_sat_shape_cen_pos_omega )
    #galaxy_cen_shape_sat_pos_omegas.append( galaxy_cen_shape_sat_pos_omega )
    #galaxy_satsat_omegas.append( galaxy_satsat_omega )
    #galaxy_cencen_omegas.append( galaxy_cencen_omega )
    #galaxy_cen_sat_etas.append( galaxy_cen_sat_eta )
    #galaxy_cen_sat_xis.append( galaxy_cen_sat_xi )
    
    #halo_diff_omegas.append( halo_omega - galaxy_omega )
    #halo_diff_etas.append( halo_eta - galaxy_eta )
    #halo_diff_xis.append( halo_xi - galaxy_xi )
    
    print(loop)

tot_time = time.time() - begin
print( tot_time, " for ", runs )
print( tot_time/runs, " per run" )

number of galaxies:  49149
1
number of galaxies:  49198
2
number of galaxies:  48812
3
number of galaxies:  48963
4
number of galaxies:  48846
5
number of galaxies:  49122
6
number of galaxies:  49098
7
number of galaxies:  48694
8
number of galaxies:  48960
9
number of galaxies:  48702
10
number of galaxies:  48855
11
number of galaxies:  49395
12
number of galaxies:  49153
13
number of galaxies:  48981
14
number of galaxies:  49121
15
number of galaxies:  48944
16
number of galaxies:  48822
17
number of galaxies:  48754
18
number of galaxies:  48926
19
number of galaxies:  48713
20
number of galaxies:  49032
21
number of galaxies:  49259
22
number of galaxies:  48672
23
number of galaxies:  48898
24
number of galaxies:  49112
25
number of galaxies:  48670
26
number of galaxies:  48835
27
number of galaxies:  48721
28
number of galaxies:  48784
29
number of galaxies:  49246
30
number of galaxies:  48743
31
number of galaxies:  48907
32
number of galaxies:  49067
33
number of galaxies:

In [7]:
##halo_omegas = np.array(halo_omegas)
#halo_cen_omegas = np.array(halo_cen_omegas)
#halo_etas = np.array(halo_etas)
#halo_xis = np.array(halo_xis)

#galaxy_omegas = np.array(galaxy_omegas)
#galaxy_etas = np.array(galaxy_etas)
#galaxy_xis = np.array(galaxy_xis)

#all_halo_omegas = np.array(all_halo_omegas)
#all_halo_etas = np.array(all_halo_etas)
#all_halo_xis = np.array(all_halo_xis)

all_galaxy_omegas = np.array(all_galaxy_omegas)
all_galaxy_etas = np.array(all_galaxy_etas)
all_galaxy_xis = np.array(all_galaxy_xis)

#halo_diff_omegas = np.array(halo_diff_omegas)
#halo_diff_etas = np.array(halo_diff_etas)
#halo_diff_xis = np.array(halo_diff_xis)

#halo_sat_shape_cen_pos_omegas = np.array( halo_sat_shape_cen_pos_omegas )
#halo_cen_shape_sat_pos_omegas = np.array( halo_cen_shape_sat_pos_omegas )
#halo_satsat_omegas = np.array( halo_satsat_omegas )
#halo_cencen_omegas = np.array( halo_cencen_omegas )
#halo_cen_sat_etas = np.array( halo_cen_sat_etas )
#halo_cen_sat_xis = np.array( halo_cen_sat_xis )

##galaxy_sat_shape_cen_pos_omegas = np.array( galaxy_sat_shape_cen_pos_omegas )
#galaxy_cen_shape_sat_pos_omegas = np.array( galaxy_cen_shape_sat_pos_omegas )
#galaxy_satsat_omegas = np.array( galaxy_satsat_omegas )
#galaxy_cencen_omegas = np.array( galaxy_cencen_omegas )
#galaxy_cen_sat_etas = np.array( galaxy_cen_sat_etas )
#galaxy_cen_sat_xis = np.array( galaxy_cen_sat_xis )

In [8]:
#halo_omega_df = pd.DataFrame( data = halo_omegas )
#halo_cen_omegas_df = pd.DataFrame( data = halo_cen_omegas )
#halo_eta_df = pd.DataFrame( data = halo_etas )
#halo_xi_df = pd.DataFrame( data = halo_xis )

#galaxy_omega_df = pd.DataFrame( data = galaxy_omegas )
#galaxy_eta_df = pd.DataFrame( data = galaxy_etas )
#galaxy_xi_df = pd.DataFrame( data = galaxy_xis )

#all_halo_omega_df = pd.DataFrame( data = all_halo_omegas )
#all_halo_eta_df = pd.DataFrame( data = all_halo_etas )
#all_halo_xi_df = pd.DataFrame( data = all_halo_xis )

all_galaxy_omega_df = pd.DataFrame( data = all_galaxy_omegas )
all_galaxy_eta_df = pd.DataFrame( data = all_galaxy_etas )
all_galaxy_xi_df = pd.DataFrame( data = all_galaxy_xis )

#diff_omega_df = pd.DataFrame( data = halo_diff_omegas )
#diff_eta_df = pd.DataFrame( data = halo_diff_etas )
#diff_xi_df = pd.DataFrame( data = halo_diff_xis )

##halo_sat_shape_cen_pos_omegas_df = pd.DataFrame( data = halo_sat_shape_cen_pos_omegas )
#halo_cen_shape_sat_pos_omegas_df = pd.DataFrame( data = halo_cen_shape_sat_pos_omegas )
#halo_satsat_omegas_df = pd.DataFrame( data = halo_satsat_omegas )
#halo_cencen_omegas_df = pd.DataFrame( data = halo_cencen_omegas )
#halo_cen_sat_etas_df = pd.DataFrame( data = halo_cen_sat_etas )
#halo_cen_sat_xis_df = pd.DataFrame( data = halo_cen_sat_xis )

##galaxy_sat_shape_cen_pos_omegas_df = pd.DataFrame( data = galaxy_sat_shape_cen_pos_omegas )
#galaxy_cen_shape_sat_pos_omegas_df = pd.DataFrame( data = galaxy_cen_shape_sat_pos_omegas )
#galaxy_satsat_omegas_df = pd.DataFrame( data = galaxy_satsat_omegas )
#galaxy_cencen_omegas_df = pd.DataFrame( data = galaxy_cencen_omegas )
#galaxy_cen_sat_etas_df = pd.DataFrame( data = galaxy_cen_sat_etas )
#galaxy_cen_sat_xis_df = pd.DataFrame( data = galaxy_cen_sat_xis )

In [9]:
directory = "CorrelationAverages/"
prefix = ""
suffix = "_a=0.826_gamma=0"

In [10]:
#halo_omega_df.to_csv(directory+prefix+"halo_omega"+suffix+".csv", index=False)
#halo_cen_omegas_df.to_csv(directory+prefix+"halo_cen_omega"+suffix+".csv", index=False)
#halo_eta_df.to_csv(directory+prefix+"halo_eta"+suffix+".csv", index=False)
#halo_xi_df.to_csv(directory+prefix+"halo_xi"+suffix+".csv", index=False)

#galaxy_omega_df.to_csv(directory+prefix+"galaxy_omega"+suffix+".csv", index=False)
#galaxy_eta_df.to_csv(directory+prefix+"galaxy_eta"+suffix+".csv", index=False)
#galaxy_xi_df.to_csv(directory+prefix+"galaxy_xi"+suffix+".csv", index=False)

#all_halo_omega_df.to_csv(directory+prefix+"all_halo_omega"+suffix+".csv", index=False)
#all_halo_eta_df.to_csv(directory+prefix+"all_halo_eta"+suffix+".csv", index=False)
#all_halo_xi_df.to_csv(directory+prefix+"all_halo_xi"+suffix+".csv", index=False)

all_galaxy_omega_df.to_csv(directory+prefix+"all_galaxy_omega"+suffix+".csv", index=False)
all_galaxy_eta_df.to_csv(directory+prefix+"all_galaxy_eta"+suffix+".csv", index=False)
all_galaxy_xi_df.to_csv(directory+prefix+"all_galaxy_xi"+suffix+".csv", index=False)

#diff_omega_df.to_csv(directory+prefix+"diff_omega"+suffix+".csv", index=False)
#diff_eta_df.to_csv(directory+prefix+"diff_eta"+suffix+".csv", index=False)
#diff_xi_df.to_csv(directory+prefix+"diff_xi"+suffix+".csv", index=False)

##halo_sat_shape_cen_pos_omegas_df.to_csv(directory+prefix+"halo_sat_shape_cen_pos_omegas"+suffix+".csv", index=False)
#halo_cen_shape_sat_pos_omegas_df.to_csv(directory+prefix+"halo_cen_shape_sat_pos_omegas"+suffix+".csv", index=False)
#halo_satsat_omegas_df.to_csv(directory+prefix+"halo_satsat_omegas"+suffix+".csv",index=False)
#halo_cencen_omegas_df.to_csv(directory+prefix+"halo_cencen_omegas"+suffix+".csv",index=False)
#halo_cen_sat_etas_df.to_csv(directory+prefix+"halo_cen_sat_etas"+suffix+".csv", index=False)
#halo_cen_sat_xis_df.to_csv(directory+prefix+"halo_cen_sat_xis"+suffix+".csv", index=False)

##galaxy_sat_shape_cen_pos_omegas_df.to_csv(directory+prefix+"galaxy_sat_shape_cen_pos_omegas"+suffix+".csv", index=False)
#galaxy_cen_shape_sat_pos_omegas_df.to_csv(directory+prefix+"galaxy_cen_shape_sat_pos_omegas"+suffix+".csv", index=False)
#galaxy_satsat_omegas_df.to_csv(directory+prefix+"galaxy_satsat_omegas"+suffix+".csv",index=False)
#galaxy_cencen_omegas_df.to_csv(directory+prefix+"galaxy_cencen_omegas"+suffix+".csv",index=False)
#galaxy_cen_sat_etas_df.to_csv(directory+prefix+"galaxy_cen_sat_etas"+suffix+".csv", index=False)
#galaxy_cen_sat_xis_df.to_csv(directory+prefix+"galaxy_cen_sat_xis"+suffix+"".csv", index=False)

In [11]:
import winsound

In [12]:
winsound.Beep(2500, 250)
winsound.Beep(1500, 250)
winsound.Beep(2500, 250)
winsound.Beep(1500, 250)